In [7]:
import torch as th
import syft as sy

from torch import nn
from torch import optim
import torch.nn.functional as F

hook = sy.TorchHook(th)

In [8]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [5]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(20):
        
        # 1) erase previous gradients (if they exist)
        opt.zero_grad()
        
        # 2) make a prediction
        pred = model(data)
        
        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()
        
        # 4) figure out which weights caused us to miss
        loss.backward()
        
        # 5) change those weights
        opt.step()
        
        # 6) print our progress
        print(loss.data)
        
train()

tensor(2.6608)
tensor(6.3069)
tensor(6.7998)
tensor(0.8875)
tensor(0.7891)
tensor(0.6889)
tensor(0.5824)
tensor(0.4726)
tensor(0.3640)
tensor(0.2662)
tensor(0.1861)
tensor(0.1266)
tensor(0.0961)
tensor(0.0712)
tensor(0.0496)
tensor(0.0380)
tensor(0.0320)
tensor(0.0243)
tensor(0.0196)
tensor(0.0161)


In [6]:
model(data)

tensor([[ 0.0400],
        [-0.0218],
        [ 1.0642],
        [ 0.9153]], grad_fn=<AddmmBackward>)

In [11]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

AttributeError: 'FixedPrecisionTensor' object has no attribute 'attr'

In [12]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:39353880864 -> alice:2280223990]
 	-> (Wrapper)>[PointerTensor | me:3719899085 -> bob:89054110037]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:54491581483 -> alice:18342859874]
 	-> (Wrapper)>[PointerTensor | me:3557103647 -> bob:40004445335]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:80875797512 -> alice:94432066965]
 	-> (Wrapper)>[PointerTensor | me:99383909053 -> bob:46662140530]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:89020676893 -> alice:3526099201]
 	-> (Wrapper)>[PointerTensor | me:16973222824 -> bob:80509315632]

In [14]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [15]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:98375104908 -> alice:15231108590]
	-> (Wrapper)>[PointerTensor | me:68917915602 -> bob:16529510691]
	*crypto provider: secure_worker*

In [16]:
encrypted_prediction = encrypted_model(encrypted_data)

In [18]:
encrypted_prediction.get().float_precision()

tensor([[ 0.0400],
        [-0.0210],
        [ 1.0630],
        [ 0.9140]])